In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def convert_yearly_to_monthly(data_path, output_folder):
    
    # Load the CSV file
    df = pd.read_csv(data_path)
    
    # Constants for oscillation
    df['c'] = 0.45 * np.pi * np.sign(df['latitude'])

    # Define tas for Summer (W) and Winter (C)
    conditions_summer = ((df['latitude'] >= 0) & df['month'].isin([6, 7, 8])) | ((df['latitude'] < 0) & df['month'].isin([1, 2, 12]))
    conditions_winter = ((df['latitude'] >= 0) & df['month'].isin([1, 2, 12])) | ((df['latitude'] < 0) & df['month'].isin([6, 7, 8]))

    df['tas(W)'] = np.where(conditions_summer, df['tas'] - 273.15, np.nan)
    df['tas(C)'] = np.where(conditions_winter, df['tas'] - 273.15, np.nan)

    # Conditions for oscillation factors
    df['fa1'] = df['tas(W)'].notna().astype(int)
    df['fa2'] = df['latitude'].between(-34, 35).astype(int)
    df['fa3'] = df['tas(C)'].notna().astype(int)
    df['fa4'] = ((df['latitude'] <= -34) | (df['latitude'] >= 35)).astype(int)
    
    # Calculations for a1 and a2 based on available temperatures
    df['a1'] = np.where(df['fa1'] == 1, (1 - np.exp(-(32.2 - df['tas(W)']) / 47.9)) * df['fa2'], 1)
    df['a2'] = np.where(df['fa3'] == 1, 0.1335 * (1 - np.exp(-(12.5 - df['tas(C)']) / 15.2)) * df['fa4'], 1)

    # Ensure a1 and a2 are at least 1
    df['a1'] = np.maximum(df['a1'], 1)
    df['a2'] = np.maximum(df['a2'], 1)

    # Calculate oscillation sum factor
    df['oscillation_sum'] = (3 * df['a1'] + 3 * df['a2'] + 6)

    # Calculate monthly EC based on oscillation
    df['EC_osc'] = df['EC'] / df['oscillation_sum']
    df['monthly_EC'] = np.where(df['fa1'] == 1, df['a1'] * df['EC_osc'], np.where(df['fa3'] == 1, df['a2'] * df['EC_osc'], df['EC_osc']))